<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/spark_level_2_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [3]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2025-12-29 09:36:23--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  10.5MB/s    in 32s     

2025-12-29 09:36:55 (16.3 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [4]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# 🚀 Spark Level 2 – Bài 2: Filter, Select, WithColumn & Execution Plan

## 🎯 Mục tiêu bài này

**Sau bài này bạn sẽ:**
>*	Hiểu Transformation vs Action
>*	Biết Spark có chạy ngay hay không
>*	Bắt đầu chạm vào shuffle (rất quan trọng)

---

## 🧪 Bài toán

**Dữ liệu giao dịch:**

|order_id|customer_id|amount|country|
|--------|-----------|------|-------|
|1|C001|120|VN|
|2|C002|80|VN|
|3|C003|200|SG|
|4|C001|50|VN|




### 📌 Bước 1 – Tạo DataFrame:

In [5]:
from pyspark.sql import functions as F

data = [
    (1, "C001", 120, "VN"),
    (2, "C002", 80, "VN"),
    (3, "C003", 200, "SG"),
    (4, "C001", 50, "VN"),
]

columns = ["order_id", "customer_id", "amount", "country"]

df = spark.createDataFrame(data, columns)
df.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
|       2|       C002|    80|     VN|
|       3|       C003|   200|     SG|
|       4|       C001|    50|     VN|
+--------+-----------+------+-------+



### 📌 Bước 2 – Transformation (KHÔNG chạy ngay)

```python
df_vn = df.filter(df.country == "VN")
df_vn_high = df_vn.filter(df_vn.amount > 100)
```

>❓ Spark đã chạy chưa?

>👉 CHƯA


In [6]:
df_vn = df.filter(df.country == "VN")
df_vn_high = df_vn.filter(df_vn.amount > 100)

### 📌 Bước 3 – Action (Spark bắt đầu chạy)

```python
df_vn_high.show()
```

#### 🧠 Nguyên lý quan trọng

|Loại|Ví dụ|
|----|-----|
|Transformation|filter, select, withColumn|
|Action|show, count, collect|

> 👉 Spark lazy execution


In [7]:
df_vn_high.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
+--------+-----------+------+-------+



### 📌 Bước 4 – Thêm cột mới

In [8]:
df2 = df.withColumn(
    "amount_category",
    F.when(df.amount >= 100, "HIGH").otherwise("LOW")
)

df2.show()

+--------+-----------+------+-------+---------------+
|order_id|customer_id|amount|country|amount_category|
+--------+-----------+------+-------+---------------+
|       1|       C001|   120|     VN|           HIGH|
|       2|       C002|    80|     VN|            LOW|
|       3|       C003|   200|     SG|           HIGH|
|       4|       C001|    50|     VN|            LOW|
+--------+-----------+------+-------+---------------+



### 📌 Bước 5 – Xem Execution Plan:

```python
df2.explain(True)
```

Result:

```code
== Physical Plan ==
*(1) Project
+- *(1) Scan ExistingRDD
```

> 👉 Chưa có shuffle → nhẹ

In [9]:
df2.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedstarwithcolumns(amount_category, CASE WHEN '`>=`(amount#2L, 100) THEN HIGH ELSE LOW END, None)]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Analyzed Logical Plan ==
order_id: bigint, customer_id: string, amount: bigint, country: string, amount_category: string
Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= cast(100 as bigint)) THEN HIGH ELSE LOW END AS amount_category#30]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Optimized Logical Plan ==
Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= 100) THEN HIGH ELSE LOW END AS amount_category#30]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Physical Plan ==
*(1) Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= 100) THEN HIGH ELSE LOW END AS amount_category#30]
+- *(1) Scan ExistingRDD[order_id#0L,custom

# 🚀 Spark Level 2 – Bài 3: groupBy, agg & SHUFFLE (cốt lõi Spark)

> **Đây là bài QUAN TRỌNG NHẤT trước khi bạn làm Spark thật sự trong CDP / Dataproc / EMR**


## 🎯 Mục tiêu bài này

**Sau bài này bạn sẽ:**
>*	Hiểu shuffle là gì (đúng bản chất)
>*	Biết vì sao groupBy rất đắt
>*	Đọc được execution plan có shuffle
>*	Biết khi nào Spark scale / khi nào chết

## 1️⃣ Bài toán

Dữ liệu order (như trước):

In [10]:
from pyspark.sql import functions as F

data = [
    (1, "C001", 120, "VN"),
    (2, "C002", 80, "VN"),
    (3, "C003", 200, "SG"),
    (4, "C001", 50, "VN"),
    (5, "C002", 70, "SG"),
]

columns = ["order_id", "customer_id", "amount", "country"]
df = spark.createDataFrame(data, columns)
df.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
|       2|       C002|    80|     VN|
|       3|       C003|   200|     SG|
|       4|       C001|    50|     VN|
|       5|       C002|    70|     SG|
+--------+-----------+------+-------+



## 2️⃣ GroupBy cơ bản

❓ Yêu cầu

👉 Tổng tiền theo customer_id

In [11]:
df_group = df.groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
)

df_group.show()

+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|       C001|         170|
|       C002|         150|
|       C003|         200|
+-----------+------------+



# 🔥 STOP – Đây là lúc SHUFFLE xảy ra

## 3️⃣ Shuffle là gì? (Hiểu đúng, không mơ hồ)

### 🧠 Định nghĩa CHUẨN:

**Shuffle** = **Spark** phải di chuyển dữ liệu giữa các executor để gom các key giống nhau về cùng 1 nơi

**Ví dụ:**
>*	Order của C001 nằm ở partition 1
>*	Order khác của C001 nằm ở partition 5
>> → Spark bắt buộc phải chuyển dữ liệu

>👉 Network + Disk + Serialize = tốn tài nguyên

### 🧩 Minh họa logic

```code
Partition 1: C001, C002
Partition 2: C003
Partition 3: C001, C002

groupBy(customer_id)
        ↓
Shuffle
        ↓
Partition A: C001
Partition B: C002
Partition C: C003
```



## 4️⃣ Xem Execution Plan (BẮT BUỘC)

Bạn sẽ thấy đoạn giống:

```code
Exchange hashpartitioning(customer_id, 200)
```

📌 Exchange = SHUFFLE

In [12]:
df_group.explain(True)

== Parsed Logical Plan ==
'Aggregate ['customer_id], ['customer_id, 'sum('amount) AS total_amount#64]
+- LogicalRDD [order_id#47L, customer_id#48, amount#49L, country#50], false

== Analyzed Logical Plan ==
customer_id: string, total_amount: bigint
Aggregate [customer_id#48], [customer_id#48, sum(amount#49L) AS total_amount#64L]
+- LogicalRDD [order_id#47L, customer_id#48, amount#49L, country#50], false

== Optimized Logical Plan ==
Aggregate [customer_id#48], [customer_id#48, sum(amount#49L) AS total_amount#64L]
+- Project [customer_id#48, amount#49L]
   +- LogicalRDD [order_id#47L, customer_id#48, amount#49L, country#50], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[customer_id#48], functions=[sum(amount#49L)], output=[customer_id#48, total_amount#64L])
   +- Exchange hashpartitioning(customer_id#48, 4), ENSURE_REQUIREMENTS, [plan_id=119]
      +- HashAggregate(keys=[customer_id#48], functions=[partial_sum(amount#49L)], output=[customer_id#48,

## 5️⃣ Vì sao shuffle NGUY HIỂM?

|Vấn đề|Hậu quả|
|------|-------|
|Nhiều dữ liệu|Chậm|
|Skew key|Executor chết|
|Network yếu|Timeout|
|Disk chậm|Spill|

> 👉 90% job Spark chậm = shuffle kém kiểm soát


## 6️⃣ Ví dụ SHUFFLE TỆ (anti-pattern)

```python
df.groupBy("country", "customer_id").count().show()
```

>❌ GroupBy nhiều cột không cần thiết

>❌ Cardinality cao → shuffle nặng


In [13]:
df.groupBy("country", "customer_id").count().show()

+-------+-----------+-----+
|country|customer_id|count|
+-------+-----------+-----+
|     VN|       C001|    2|
|     VN|       C002|    1|
|     SG|       C002|    1|
|     SG|       C003|    1|
+-------+-----------+-----+



## 7️⃣ Giảm shuffle – cách đầu tiên (cơ bản)

### ✅ Chỉ groupBy đúng thứ cần

```python
df.groupBy("country").sum("amount").show()
```

In [14]:
df.groupBy("country").sum("amount").show()

+-------+-----------+
|country|sum(amount)|
+-------+-----------+
|     VN|        250|
|     SG|        270|
+-------+-----------+



## 8️⃣ Kiểm soát số partition khi shuffle

In [15]:
# Mặc định:

spark.conf.get("spark.sql.shuffle.partitions")

# → thường là 200 (QUÁ NHIỀU cho dataset nhỏ)

'4'

In [16]:
# 🔧 Giảm xuống khi test / small data

spark.conf.set("spark.sql.shuffle.partitions", "4")

# 👉 Chạy lại groupBy và explain